In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

import os
os.chdir('..')

In [5]:
from langchain_openai import ChatOpenAI
from langchain_zhipu import ChatZhipuAI
from illufly.md import create_agent

In [6]:
app = create_agent(llm=ChatZhipuAI())

In [7]:
app.invoke({"task": "帮我创作一篇500字的都市修仙故事"})


 -------------------- 已有outline: 0

 -------------------- 已有detail: 0
### 反思
当前没有写作提纲，也没有已完成的扩写内容。因此，我应当首先创作写作提纲。

### 思考
下一步我需要选择`create_outline`工具来创作写作提纲。该提纲需要包含足够的细节，以便之后扩写500字的都市修仙故事。

### 推理
我需要构思一个包含以下元素的提纲：
- 主人公介绍
- 都市背景描述
- 修仙元素的引入
- 冲突或挑战
- 解决方案或结局

### 计划
以下是我的执行计划：

```json
{"properties": {"name": "create_outline", "args": {"task": "创作一个包含主人公介绍、都市背景、修仙元素、冲突和解决方案的提纲"}}}
```

这个计划聚焦于创作一个结构化的提纲，确保之后的故事扩写有明确的指导。只计划了这一步，因为一个好的提纲是成功扩写故事的基础。

### 输出
以下是我选择的执行动作：

```json
{"name": "create_outline", "args": {"task": "创作一个包含主人公介绍、都市背景、修仙元素、冲突和解决方案的提纲"}}
```

KeyboardInterrupt: 

In [ ]:
for x in app.stream({"task": "帮我创作一篇2000字的都市修仙故事"}):
    print(x)

In [ ]:
async for x in app.astream_events({"task": "帮我创作一篇2000字的都市修仙故事"}, version="v1"):
    kind = x["event"]
    if kind == "on_chat_model_stream":
        content = x["data"]["chunk"].content
        if content:
            print(content, end="", flush=True)
    else:
        # print(kind)
        pass

## 可视化

In [ ]:
%%capture --no-stderr
%pip install pygraphviz

In [ ]:
app.get_graph().print_ascii()

In [ ]:
from langchain_core.runnables.graph import MermaidDrawMethod
from IPython.display import display, Image

display(
    Image(app.get_graph().draw_mermaid_png(draw_method=MermaidDrawMethod.API))
)

## RAG

In [ ]:
from langchain_zhipu import ChatZhipuAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain.schema import Document
from langchain.agents import tool

from illufly.agents import create_tools_calling_executor
from illufly.qa import AskDocumentTool, create_qa_chain

query_document = Document(page_content="illufly 是一个基于AI的开源框架，用于生成超长文档")

db = FAISS.from_documents([query_document], OpenAIEmbeddings())

llm_zhipu = ChatZhipuAI()

chain = create_qa_chain(llm_zhipu, db.as_retriever())
qa_tool = AskDocumentTool(chain=(chain | StrOutputParser()), name="qa_chain")

In [ ]:
chain.invoke({"query": "illufly是什么"})

In [ ]:
chain.get_graph().print_ascii()

In [ ]:
from langchain.tools import tool
import random

# @tool(args_schema=WhereIsCatSchema)
@tool("WhereIsCatHidding")
def where_is_cat_hiding(idea: str) -> str:
    """从这些地方选择猫躲藏的地方"""
    return random.choice(["在床底下", "在书架中", "在阳台"])

In [ ]:
# from langchain_openai import ChatOpenAI
# llm_zhipu = ChatZhipuAI()
tools = [qa_tool, where_is_cat_hiding]
app = create_tools_calling_executor(llm_zhipu, tools)

In [ ]:
for x in app.stream("讲个笑话给我?"):
    print(x)

In [ ]:
from langchain_core.messages import HumanMessage
app = create_tools_calling_executor(
    llm_zhipu,
    tools = tools,
    verbose = True)

input = "请查询资料，告诉我illufly是什么？"
async for chunk in app.astream_events(input, version="v1"):
    # print(" "*10, chunk['event'], chunk['name'], chunk['tags'])
    if(chunk['event']=="on_chain_stream" and chunk['name'] in ["agent", "qa_chain"]):
        print(chunk['data']['chunk'].content, end="_", flush=True)
    elif(chunk['name']=="action"):
        print(chunk['data'])
    elif(chunk['event']=="on_chat_model_start"):
	    print("\n", chunk['event'], chunk['name'], chunk['tags'])        

In [ ]:
app.get_graph().print_ascii()

In [ ]:
app = create_tools_calling_executor(
    llm_zhipu,
    tools = tools,
    runnables = {"qa_chain": chain},
    verbose = True
)

input = ["请查询资料，告诉我illufly是什么？"]
async for chunk in app.astream_events(input, version="v1"):
    # print(" "*10, chunk['event'], chunk['name'], chunk['tags'])
    if(chunk['event']=="on_chain_stream" and chunk['name'] in ["agent", "qa_chain"]):
        print(chunk['data']['chunk'].content, end="_", flush=True)
    elif(chunk['name']=="action"):
        print(chunk['data'])
    elif(chunk['event']=="on_chat_model_start"):
	    print("\n", chunk['event'], chunk['name'], chunk['tags'])

In [ ]:
app.get_graph().print_ascii()

In [ ]:
app = create_tools_calling_executor(
    llm_zhipu,
    tools = tools,
    runnables = {"qa_chain": {"node": chain, "to": "agent"}},
    verbose = True)
app.get_graph().print_ascii()

In [ ]:
inputs = ["请查询资料，告诉我langchain_chinese是什么？"]
async for chunk in app.astream_events(inputs, version="v1"):
    # print(" "*10, chunk['event'], chunk['name'], chunk['tags'])
    if(chunk['event']=="on_chain_stream" and chunk['name'] in ["agent", "qa_chain"]):
        print(chunk['data']['chunk'].content, end="_", flush=True)
    elif(chunk['name']=="action"):
        print(chunk['data'])
    elif(chunk['event']=="on_chat_model_start"):
	    print("\n", chunk['event'], chunk['name'], chunk['tags'])

In [ ]:
inputs = [HumanMessage(content="猫在哪里？")]
async for chunk in app.astream_events(inputs, version="v1"):
    # print(" "*10, chunk['event'], chunk['name'], chunk['tags'])
    if(chunk['event']=="on_chain_stream" and chunk['name'] in ["agent", "qa_chain"]):
        print(chunk['data']['chunk'].content, end="_", flush=True)
    elif(chunk['name']=="action"):
        print(chunk['data'])

In [ ]:
inputs =  [HumanMessage(content="霍金的生日是哪一天？")]
async for chunk in app.astream_events(inputs, version="v1"):
    # print(" "*10, chunk['event'], chunk['name'], chunk['tags'])
    if(chunk['event']=="on_chain_stream" and chunk['name']=="agent"):
        print(chunk['data']['chunk'].content, end="_", flush=True)
    elif(chunk['name']=="action"):
        print(chunk['data'])

In [ ]:
inputs =  [HumanMessage(content="写一个关于程序员的笑话")]
async for chunk in app.astream_events(inputs, version="v1"):
    # print(" "*10, chunk['event'], chunk['name'], chunk['tags'])
    if(chunk['event']=="on_chain_stream" and chunk['name']=="agent"):
        print(chunk['data']['chunk'].content, end="_", flush=True)
    elif(chunk['name']=="action"):
        print(chunk['data'])